In [1]:
import requests
from bs4 import BeautifulSoup
import time
import xlsxwriter as xlsxwriter
import csv
import math
import re
from datetime import datetime
import numpy as np
import pandas as pd

In [5]:
movie_raw = pd.read_csv("../Data/train/data_ver4.csv", dtype={'관객수':np.str_, '스크린수':np.str_, '상영횟수':np.str_})
movie_raw.head()

,영화명,개봉일,매출액,누적매출액,관객수,누적관객수,스크린수,상영횟수,대표국적,배급사,등급,장르,감독,배우
0,월드 워 핵전쟁의 서막,2019-03-07,26000,26000,17,17,5,17,뉴질랜드,주식회사 풍경소리,15세이상관람가,액션,A.K. 스트롬,NaN
1,울프라이징,2016-11-17,300000,300000,60,60,1,1,미국,주식회사 루믹스미디어,15세이상관람가,공포(호러),BC 퍼트니,빌 오버스트 주니어
2,제3의앵글 : 형형색색대만전,NaN,1830000,1830000,305,305,1,2,대만,NaN,NaN,애니메이션,BITO,NaN
3,잃어버린 세계를 찾아 2015,2015-12-03,4000,4000,1,1,1,1,미국,(주)케이알씨지,12세이상관람가,어드벤처,C. 토마스 하우웰,"C. 토마스 하우웰,티모시 바톰즈"
4,과부: 탐스러운 육체,2016-11-19,5000,5000,1,1,1,1,일본,(주)영진크리에이티브,청소년관람불가,멜로/로맨스,C.나가후쿠,"타치바나 케이코,이치죠 리노"


In [42]:
def main():
    # 파일 불러오기
    movie_raw = pd.read_csv("../Data/train/data_ver4.csv", dtype={'관객수':np.str_, '스크린수':np.str_, '상영횟수':np.str_})
    movie_raw = movie_raw.loc[:, "영화명": "배우"]
    movie = movie_raw

    #  데이터 추출 후 Nan을 빈값으로 바꿔줌
    movie = movie.fillna('')
    movie = movie[9571:10000]

    # 엑셀파일 만들기
    outWorkbook = xlsxwriter.Workbook("Distributor"+ datetime.today().strftime("%Y%m%d%H%M%S")+".xlsx")
    worksheet = outWorkbook.add_worksheet("Distributor")

    #엑셀의 해더를 만든다.
    excel_export_head(worksheet)


    # 엑셀 1행부터 시작하기 위해 선언
    cnt = 1

    # 예외처리 후 프로그램이 돌다 죽을 경우를 대비해서 엑셀파일 생성 여부를 판단하는 구분자.
    excel_yn = False
    try:

        # CSV 파일에서 읽은 데이터 프레임을 행별로 뽑는다.
        for index,row in movie.iterrows():

            distribution = row['배급사']
            grade = row['등급']
            genre= row['장르']
            actor = row['배우']

            # 배급사, 등급, 장르, 배우가 모두 있는경우 크롤링 안함.
            if( len(distribution) > 0 and len(grade) > 0  and len(genre) > 0 and len(actor) > 0):
                movie_dic = {}
                movie_dic['배급사'] = distribution
                movie_dic['등급'] = grade
                movie_dic['장르'] = genre
                movie_dic['배우'] = actor

                print("전처리데이터:", cnt, row['영화명'], " -> ", movie_dic)
                print("--------------------------------------------------")
                # 엑셀파일 생성
                excel_export(worksheet, cnt, row, movie_dic)

            else:

                # 영화리스트에서 해당영화를 찾고 데이터 딕셔너리를 리턴한다.
                movie_dic = movie_search_list(row)

                # 받은 데이터 딕셔너리를 엑셀에 출력한다.
                excel_export(worksheet, cnt, row, movie_dic)
                print("크롤링데이터:",cnt , row['영화명']," -> " , movie_dic)
                print("--------------------------------------------------")

            # 다음행을 가르키기 위해 하나씩 더한다.
            cnt += 1
    except Exception as e:  # 모든 예외의 에러 메시지를 출력할 때는 Exception을 사용
        cnt += 1 # 행번호 증가
        print("예외처리된 영화명 :", cnt, row['영화명'])
        print('예외가 발생했습니다. 예외처리 내용', e)
        print("--------------------------------------------------")
        excel_yn = True # 예외처리후 엑셀을 다시 닫는걸 방지.
        outWorkbook.close() # 엑셀 닫고 생성

    if(excel_yn == False):
        # 엑셀파일을 다 쓴후 닫는다.
        outWorkbook.close()

In [7]:
def excel_export_head(worksheet):

    worksheet.write(0, 0, "영화명")
    worksheet.write(0, 1, "개봉일")
    worksheet.write(0, 2, "매출액")
    worksheet.write(0, 3, "누적매출액")
    worksheet.write(0, 4, "관객수")
    worksheet.write(0, 5, "누적관객수")
    worksheet.write(0, 6, "스크린수")
    worksheet.write(0, 7, "상영횟수")
    worksheet.write(0, 8, "대표국적")
    worksheet.write(0, 9, "배급사")
    worksheet.write(0, 10, "등급")
    worksheet.write(0, 11, "장르")
    worksheet.write(0, 12, "감독")
    worksheet.write(0, 13, "배우")

In [8]:
# 엑셀 데이터 쓰기
def excel_export(worksheet,cnt, row,movie_dic):
    worksheet.write(cnt, 0, row['영화명'])
    worksheet.write(cnt, 1, row["개봉일"])
    worksheet.write(cnt, 2, row["매출액"])
    worksheet.write(cnt, 3, row["누적매출액"])
    worksheet.write(cnt, 4, row["관객수"])
    worksheet.write(cnt, 5, row["누적관객수"])
    worksheet.write(cnt, 6, row["스크린수"])
    worksheet.write(cnt, 7, row["상영횟수"])
    worksheet.write(cnt, 8, row["대표국적"])
    worksheet.write(cnt, 9, movie_dic["배급사"])
    worksheet.write(cnt, 10, movie_dic["등급"])
    worksheet.write(cnt, 11, movie_dic["장르"])
    worksheet.write(cnt, 12, row["감독"])
    worksheet.write(cnt, 13, movie_dic["배우"])

In [9]:
# 네이버 영화리스트 페이지에서 찾고자 하는 영화를 찾는다.
def movie_search_list(row):
    movie_dic = {}
    movie_dic['배급사'] = ""
    movie_dic['등급'] = ""
    movie_dic['장르'] = ""
    movie_dic['배우'] = ""
    movie_name = row['영화명']
    director_name = row["감독"]

    time.sleep(1)

    # 네이버 영화 페이지에서 영화명 검색
    url = 'https://movie.naver.com/movie/search/result.naver?section=movie&query=' + str(movie_name)
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'lxml', from_encoding='utf-8')

    # 검색되는 영화이름 모두 찾기
    # ul 태그의 클래스 search_list_1 찾음
    items = soup.find("ul", {"class": "search_list_1"})

    # 검색되는 영화가 있을 경우 확인
    if (items != None):
        # ul 태그 아래의 li 태그를 모두 찾음
        li_items = items.find_all('li')

        # 결과값을 담을 리스트 생성
        item_list = []
        for li_item in li_items:

            # 결과값을 담을 딕셔너리 생성
            item_dict = {}

            # 영화제목, URL, 감독 값을 태그에서 찾아 딕셔너리
            # 감독명과 타이틀에서 공백을 제거하고 리스트에 넣는다.
            name_tag = li_item.find('dt')
            name = name_tag.text
            if (name.find('(') > -1):
                name = ''.join(name[:name.find('(') - 1].split())  # 모든 공백제거
                item_dict['타이틀'] = name
            else:
                name = ''.join(name.split())  # 모든 공백제거
                item_dict['타이틀'] = name

            movie_url = name_tag.find('a')['href']
            item_dict['주소'] = movie_url
            dd_items = li_item.find_all("dd", {"class": "etc"})

            for dd_item in dd_items:
                dd_tag = dd_item.text

                if (dd_tag.find('감독') != -1):
                    if (dd_tag.find('|') > -1):
                        director_tag = dd_tag[:dd_tag.find('|')]
                        director = str(director_tag).replace('감독 : ', "") # 감독 단어제거
                        director = ''.join(director.split()) # 모든 공백제거
                        item_dict['감독'] = director
                    else:
                        director = dd_tag.replace('감독 : ', "")
                        director = ''.join(director.split())
                        item_dict['감독'] = director

            # 리스트에 딕셔너리를 넣는다
            item_list.append(item_dict)

        #  네이버에서 가져온 영화정보를 CSV 파일에서 가져온 값과 비교를 해서 정말 그 영화인지 찾는다.
        for naver_item in item_list:

            title = naver_item.get('타이틀')
            director = naver_item.get('감독')
            url = str(naver_item.get('주소')).replace('/movie/bi/mi/basic.naver?code=', '')

            director_name =  ''.join(str(director_name).split()) # 전처리 데이터의 감독명 공백제거
            movie_name = ''.join(str(movie_name).split()) # 전처리 데이터 영화이름 공백제거

            if(director != None):
                director_ratio = name_match_ratio(director,director_name) # 크롤링한 감독명과 전처리 데이터의 단어일치율 계산
            else:
                director_ratio = 0 # 크롤링 한 대상에 감독명이 없을 경우 0%

            if(title != None):
                movie_name_ratio = name_match_ratio(title,movie_name) # 크롤링한 감독명과 전처리 데이터의 단어일치율 계산
            else:
                movie_name_ratio = 0 # 크롤링 한 대상에 감독명이 없을 경우 0%


            if (director_ratio >= 60 and movie_name_ratio == 100):

                url = 'https://movie.naver.com/movie/bi/mi/detail.naver?code=' + str(url)
                response = requests.get(url)
                soup = BeautifulSoup(response.content, "lxml", from_encoding="utf-8")

                movie_dic['배급사'] = movie_distribution(soup) # 배급사 찾기함수
                movie_dic['등급'] = movie_grade(soup) # 등급찾기 함수
                movie_dic['장르'] = movie_genre(soup) # 장르찾기 함수
                movie_dic['배우'] = movie_actor(soup) # 배우찾기

                print(' -- 감독명 : ', director, "||", director_name, '-->  일치율: ' ,director_ratio, "%" )
                print(' -- 영화이름 : ', movie_name, "||", title,'-->  일치율: ' ,movie_name_ratio, "%")
                print(' -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-')
            else:

                print(' -- 감독명 : ',director,"||",director_name, '--> 일치율: ' ,director_ratio, "%" )
                print(' -- 영화이름 : ', movie_name, "||", title ,'--> 일치율: ' ,movie_name_ratio, "%" )
                print(' -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-')
                
    return movie_dic

In [10]:
# 두 단어간 일치 비율을 리턴한다.
def name_match_ratio(word1,word2):

    d1 = list(filter(None,list(word1))) # 공백문자제거
    d1 = list(set(d1).intersection(d1)) # 중복되는 글자 원소 한개만 남김
    d2 = list(filter(None,list(word2))) # 공백문자제거
    d2 = list(set(d2).intersection(d2)) # 중복되는 글자 원소 한개만 남김
    d3 = list(set(d1).intersection(d2)) # 서로 동일한 단어를 리스트화
    if(word1 == word2):
        ratio = 100 #두단어를 비교했는데 정확하게 일치하면 100%
    else:
        # 두단어를 비교했는데 같은 글자의 갯수를 찾고
        ratio = int(len(d3) / len(d1) * 100) # 두 리스트 원소갯수를 비교하여 백분율로 변환
        print(d1,"::",d2,"-->",d3)

    return ratio

In [11]:
# 배급사정보를 html에서 찾는다.
def movie_distribution(soup):
    distribution = ''

    items = soup.find("dl", {"class": "agency_name"})
    # 배급사 정보가 없을 경우 체크
    if (items != None):

        dt_items = items.find_all('dt')
        dd_items = items.find_all('dd')
        for idx, i in enumerate(dt_items):
            # 배급사 정보 체크
            if (i.text == '배급'):
                distribution = str(dd_items[idx].text).replace('\n', '').replace('\t','').replace('\r','')
    return distribution

In [12]:
# 등급을 html에서 찾는다.
def movie_grade(soup):
    grade = ''

    items = soup.find("dl", {"class": "info_spec"})
    # 등급 정보가 없을 경우 체크
    if (items != None):

        dt_items = items.find_all('dt')
        dd_items = items.find_all('dd')
        for idx, i in enumerate(dt_items):
            # 등급 정보 체크
            if (i.text == '등급'):
                grad_line = str(dd_items[idx].text).replace('\n', '').replace('\r', '').replace('\t', '')
                if(grad_line.find('[해외]')> -1):
                    grade = grad_line[:grad_line.find('[해외]')].replace('[국내] ','')
                else:
                    grade = grad_line
    return grade

In [13]:
def movie_genre(soup):
    genre = ''

    items = soup.find("dl", {"class": "info_spec"})
    # 장르 정보가 없을 경우 체크
    if (items != None):

        dt_items = items.find_all('dt')
        dd_items = items.find_all('dd')
        for idx, i in enumerate(dt_items):
            # 장르 정보 체크
            if (str(i.text).find('개요') > -1):
                dd_spans = dd_items[idx].find_all('span')
                genre = str(dd_spans[0].text).replace('\n', '').replace('\r', '').replace('\t', '').split(',')[0]
    return genre

In [14]:
def movie_actor(soup):
    actor = ''
    actor_namelist = []
    items = soup.find("ul", {"class": "lst_people"})
    # 배우 정보가 없을 경우 체크
    if (items != None):
        # 모든 배우이름을 찾는다.
        a_items = items.find_all('a', {"class": "k_name"})
        # 배우이름을 담을 리스트를 만든다.

        # 찾은 모든 배우를 찾아 리스트에 담는다.
        for idx, i in enumerate(a_items):
            # 배우 정보 체크
            actor_name = str(i.text)
            actor_namelist.append(actor_name)
            # 4번째부터는 안넣음.
            if(idx == 2):
                break
    if actor_namelist != None:
        actor = ",".join(actor_namelist)

    return actor

In [43]:
if __name__ == '__main__':
    main()

 -- 감독명 :  알렉산더소쿠로프 || 알렉산더소쿠로프 -->  일치율:  100 %
 -- 영화이름 :  일식의날 || 일식의날 -->  일치율:  100 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
크롤링데이터: 1 일식의 날  ->  {'배급사': '', '등급': '15세 관람가 ', '장르': '드라마', '배우': '알렉세이 아나니시노프,에스켄더 우마로프,블라디미르 자만스키'}
--------------------------------------------------
['등', '화', '도'] :: ['산', '렉', '더', '알', '소', '로', '쿠', '프'] --> []
[':', '리', '아', '자', '빠', '우', '황', '소'] :: ['자', '리', '황', '소'] --> ['자', '황', '리', '소']
 -- 감독명 :  등화도 || 알렉산더소쿠로프 --> 일치율:  0 %
 -- 영화이름 :  황소자리 || 황소자리:우리아빠 --> 일치율:  50 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
크롤링데이터: 2 황소자리  ->  {'배급사': '', '등급': '', '장르': '', '배우': ''}
--------------------------------------------------
['레', '르', '리', '이', '도', '아', '헬', '나', '마', '스', '키', '에', ',', '시', '가'] :: ['산', '렉', '더', '알', '소', '로', '쿠', '프'] --> []
 -- 감독명 :  마리나이스키에도가르시아,헬레나이스키에도가르시아 || 알렉산더소쿠로프 --> 일치율:  0 %
 -- 영화이름 :  태양 || 태양 --> 일치율:  100 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
['유', '에', '리', '이'] :: ['산', '렉', '더', '알', '소', '로', '쿠', '프'] --> [

 -- 감독명 :  알렉산더소쿠로프 || 알렉산더소쿠로프 -->  일치율:  100 %
 -- 영화이름 :  알렉산드라 || 알렉산드라 -->  일치율:  100 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
['드', '주', '라'] :: ['산', '렉', '더', '알', '소', '로', '쿠', '프'] --> []
 -- 감독명 :  라드주드 || 알렉산더소쿠로프 --> 일치율:  0 %
 -- 영화이름 :  알렉산드라 || 알렉산드라 --> 일치율:  100 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
['클', '너', '샤', '린', '랭', 'J', '.', '프'] :: ['산', '렉', '더', '알', '소', '로', '쿠', '프'] --> ['프']
['산', '렉', '콜', '스', '알', '라', '와', '니', '드'] :: ['산', '렉', '알', '라', '드'] --> ['산', '렉', '알', '라', '드']
 -- 감독명 :  프랭클린J.샤프너 || 알렉산더소쿠로프 --> 일치율:  12 %
 -- 영화이름 :  알렉산드라 || 니콜라스와알렉산드라 --> 일치율:  55 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
['모', '리', '센', '나', '스', '베', '유', '테', '프'] :: ['산', '렉', '더', '알', '소', '로', '쿠', '프'] --> ['프']
['미', '산', '아', '렉', '알', '라', '와', '드'] :: ['산', '렉', '알', '라', '드'] --> ['산', '렉', '알', '라', '드']
 -- 감독명 :  모나프리스베테유센 || 알렉산더소쿠로프 --> 일치율:  11 %
 -- 영화이름 :  알렉산드라 || 미아와알렉산드라 --> 일치율:  62 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
['어', '롤', '히', '드', '프'] :: ['산', 

크롤링데이터: 35 병기고  ->  {'배급사': '', '등급': '', '장르': '', '배우': ''}
--------------------------------------------------
['산', '르', '도', '렉', '첸', '알', '코', '드', '브'] :: ['산', '르', '도', '렉', '알', '젠', '코', '드', '브'] --> ['드', '산', '르', '렉', '알', '코', '도', '브']
 -- 감독명 :  알렉산드르도브첸코 || 알렉산드르도브젠코 -->  일치율:  88 %
 -- 영화이름 :  즈베니고라 || 즈베니고라 -->  일치율:  100 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
크롤링데이터: 36 즈베니고라  ->  {'배급사': '', '등급': '', '장르': '90분 ', '배우': ''}
--------------------------------------------------
['버', '즈', '자', '네', '거'] :: ['산', '르', '도', '렉', '알', '젠', '코', '드', '브'] --> []
 -- 감독명 :  자네즈버거 || 알렉산드르도브젠코 --> 일치율:  0 %
 -- 영화이름 :  이반 || 이반 --> 일치율:  100 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
['다', '나', '알', '바', '비', '도', '료'] :: ['산', '르', '도', '렉', '알', '젠', '코', '드', '브'] --> ['도', '알']
 -- 감독명 :  알료나다비도바 || 알렉산드르도브젠코 --> 일치율:  28 %
 -- 영화이름 :  이반 || 이반 --> 일치율:  100 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
['산', '르', '도', '렉', '첸', '알', '코', '드', '브'] :: ['산', '르', '도', '렉', '알', '젠', '코', '드'

['카', '이', '야', '라', '시', '즈'] :: ['튜', '매', '르', '리', '텔', '에', '포', '렉', '상', '파', '델', '알', '라', '드', ',', '테'] --> ['라']
['새', '이', '는', '없', '름'] :: ['그', '이', '은', '.', '름'] --> ['이', '름']
 -- 감독명 :  시라이시카즈야 || 알렉상드르드라파텔리에르,매튜델라포르테 --> 일치율:  16 %
 -- 영화이름 :  그이름은... || 이름없는새 --> 일치율:  40 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
['그', '이', '…', '은', '의', '름'] :: ['그', '이', '은', '.', '름'] --> ['그', '은', '이', '름']
 -- 감독명 :  None || 알렉상드르드라파텔리에르,매튜델라포르테 --> 일치율:  0 %
 -- 영화이름 :  그이름은... || 그의이름은… --> 일치율:  66 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
['김', '석', '환'] :: ['튜', '매', '르', '리', '텔', '에', '포', '렉', '상', '파', '델', '알', '라', '드', ',', '테'] --> []
['버', '그', '레', '이', '트', '은', '의', '름'] :: ['그', '이', '은', '.', '름'] --> ['그', '은', '이', '름']
 -- 감독명 :  김석환 || 알렉상드르드라파텔리에르,매튜델라포르테 --> 일치율:  0 %
 -- 영화이름 :  그이름은... || 그의이름은트레버 --> 일치율:  50 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
['범', '철', '권'] :: ['튜', '매', '르', '리', '텔', '에', '포', '렉', '상', '파', '델', '알', '라', '드', ',', '테'] --> []
['버', '그', 

 -- 감독명 :  알렉스드라이글레시아 || 알렉스드라이글레시아 -->  일치율:  100 %
 -- 영화이름 :  더바 || 더바 -->  일치율:  100 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
['이', '리', '슬', '언', '라', '워'] :: ['레', '글', '이', '아', '렉', '스', '알', '라', '드', '시'] --> ['이', '라']
['이', '징', '더', '바', '라'] :: ['더', '바'] --> ['더', '바']
 -- 감독명 :  라이언워슬리 || 알렉스드라이글레시아 --> 일치율:  33 %
 -- 영화이름 :  더바 || 라이징더바 --> 일치율:  40 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
['슨', '헤', '이', '리', '들', '제'] :: ['레', '글', '이', '아', '렉', '스', '알', '라', '드', '시'] --> ['이']
['더', '앳', '바'] :: ['더', '바'] --> ['더', '바']
 -- 감독명 :  제이슨헤들리 || 알렉스드라이글레시아 --> 일치율:  16 %
 -- 영화이름 :  더바 || 앳더바 --> 일치율:  66 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
['리', '퍼', '스', '크', '토', '로', '메'] :: ['레', '글', '이', '아', '렉', '스', '알', '라', '드', '시'] --> ['스']
['어', '툴', '이', '더', '스', '바', '세', '비'] :: ['더', '바'] --> ['더', '바']
 -- 감독명 :  크리스토퍼로메로 || 알렉스드라이글레시아 --> 일치율:  14 %
 -- 영화이름 :  더바 || 더바스툴세이비어 --> 일치율:  25 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
['리', '찰', '린', '채', '플'] :: ['레', '글', '이', '아', '

...

 -- 감독명 :  앤드류존스 || 앤드류존스 -->  일치율:  100 %
 -- 영화이름 :  1944:인간병기 || 1944:인간병기 -->  일치율:  100 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
크롤링데이터: 303 1944: 인간병기  ->  {'배급사': '', '등급': '[국내] 15세 관람가', '장르': '액션', '배우': ''}
--------------------------------------------------
 -- 감독명 :  앤드류존스 || 앤드류존스 -->  일치율:  100 %
 -- 영화이름 :  알카트라즈:죽음의수용소 || 알카트라즈:죽음의수용소 -->  일치율:  100 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
크롤링데이터: 304 알카트라즈: 죽음의 수용소  ->  {'배급사': '', '등급': '[국내] 15세 관람가', '장르': '액션', '배우': ''}
--------------------------------------------------
전처리데이터: 305 폴터가이스트 액티비티  ->  {'배급사': '(주)피터팬픽쳐스', '등급': '청소년관람불가', '장르': '공포(호러)', '배우': '리 베인'}
--------------------------------------------------
 -- 감독명 :  앤드류존스 || 앤드류존스 -->  일치율:  100 %
 -- 영화이름 :  킬러넥스트도어 || 킬러넥스트도어 -->  일치율:  100 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
크롤링데이터: 306 킬러 넥스트도어  ->  {'배급사': '', '등급': '[국내] 15세 관람가', '장르': '스릴러', '배우': '데렉 넬슨,나이젤 바버,윌리엄 메러디스'}
--------------------------------------------------
 -- 감독명 :  앤드류존스 || 앤드류존스 -->  일치율:  

['카', '이', '히', '치', '라', '와', '로', '유'] :: ['후', '앤', '앙', '류', '드'] --> []
['이', '리', '나', '는', '만', '없', '거'] :: ['만', '나', '이'] --> ['나', '만', '이']
 -- 감독명 :  히라카와유이치로 || 앤드류후앙 --> 일치율:  0 %
 -- 영화이름 :  나만이 || 나만이없는거리 --> 일치율:  42 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
 -- 감독명 :  앤드류후앙 || 앤드류후앙 -->  일치율:  100 %
 -- 영화이름 :  나만이 || 나만이 -->  일치율:  100 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
['모', '이', '히', '토', '코'] :: ['후', '앤', '앙', '류', '드'] --> []
['이', '리', '나', '는', '만', '없', '거'] :: ['만', '나', '이'] --> ['나', '만', '이']
 -- 감독명 :  이토토모히코 || 앤드류후앙 --> 일치율:  0 %
 -- 영화이름 :  나만이 || 나만이없는거리 --> 일치율:  42 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
['지', '오', '르', '티', '마', '노', '세'] :: ['후', '앤', '앙', '류', '드'] --> []
['악', '열', '쇠', '를', '만', '실', '은', '죄', '너', '다', '나', '있', '밀', ',', '의', '지', '오', '이', '직', '고', '가'] :: ['만', '나', '이'] --> ['나', '만', '이']
 -- 감독명 :  세르지오마르티노 || 앤드류후앙 --> 일치율:  0 %
 -- 영화이름 :  나만이 || 너의죄악은밀실,오직나만이열쇠를가지고있다 --> 일치율:  14 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
크롤링데이터: 323 

 -- 감독명 :  앨리스넬리스 || 앨리스넬리스 -->  일치율:  100 %
 -- 영화이름 :  리틀걸블루 || 리틀걸블루 -->  일치율:  100 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
크롤링데이터: 351 리틀 걸 블루  ->  {'배급사': '', '등급': '', '장르': '드라마', '배우': '카렐 로든,이바 비토바,이반 프래넥'}
--------------------------------------------------
전처리데이터: 352 파라다이스 힐스  ->  {'배급사': '(주)올스타엔터테인먼트', '등급': '12세이상관람가', '장르': '판타지', '배우': '엠마 로버츠,밀라 요보비치,에이자 곤잘레스,아콰피나,다니엘 맥도널드'}
--------------------------------------------------
전처리데이터: 353 매릴랜드  ->  {'배급사': 'THE 픽쳐스', '등급': '15세이상관람가', '장르': '액션', '배우': '마티아스 쇼에나에츠,다이앤 크루거'}
--------------------------------------------------
전처리데이터: 354 프록시마 프로젝트  ->  {'배급사': '(주)영화사 진진', '등급': '12세이상관람가', '장르': '드라마', '배우': '에바 그린,맷 딜런'}
--------------------------------------------------
전처리데이터: 355 생텀  ->  {'배급사': '(주)시너지하우스 (시너지)', '등급': '15세이상관람가', '장르': '어드벤처', '배우': '이안 그루퍼드,리차드 록스버그,라이스 웨이크필드,댄 와일리'}
--------------------------------------------------
전처리데이터: 356 이차크의 행복한 바이올린  ->  {'배급사': '(주)영화사 진진', '등급': '전체관람가', '장르': '다큐멘터리', '배우':

 -- 감독명 :  야네스헨드릭즈 || 야네스헨드릭즈 -->  일치율:  100 %
 -- 영화이름 :  앨리스매드니스리턴즈 || 앨리스매드니스리턴즈 -->  일치율:  100 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
크롤링데이터: 375 앨리스 매드니스 리턴즈  ->  {'배급사': '', '등급': '', '장르': '애니메이션', '배우': ''}
--------------------------------------------------
크롤링데이터: 376 엠넷 개똥벌레편  ->  {'배급사': '', '등급': '', '장르': '', '배우': ''}
--------------------------------------------------
크롤링데이터: 377 엠넷 무당벌레편  ->  {'배급사': '', '등급': '', '장르': '', '배우': ''}
--------------------------------------------------
 -- 감독명 :  야네스헨드릭즈 || 야네스헨드릭즈 -->  일치율:  100 %
 -- 영화이름 :  유나이티드항공-해양의오케스트라 || 유나이티드항공-해양의오케스트라 -->  일치율:  100 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
크롤링데이터: 378 유나이티드 항공 - 해양의 오케스트라  ->  {'배급사': '', '등급': '', '장르': '애니메이션', '배우': ''}
--------------------------------------------------
크롤링데이터: 379 프리스키즈 어드벤처랜드  ->  {'배급사': '', '등급': '', '장르': '', '배우': ''}
--------------------------------------------------
 -- 감독명 :  야네스헨드릭즈 || 야네스헨드릭즈 -->  일치율:  100 %
 -- 영화이름 :  클로버-웨이베터 || 클로버-웨이베터 -->  일치율:  100 

 -- 감독명 :  야마가타카유키 || 야마가타카유키 -->  일치율:  100 %
 -- 영화이름 :  신배틀로얄:극악소년원 || 신배틀로얄:극악소년원 -->  일치율:  100 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
크롤링데이터: 398 신 배틀로얄:극악 소년원  ->  {'배급사': '', '등급': '[국내] 청소년 관람불가', '장르': '액션', '배우': ''}
--------------------------------------------------
 -- 감독명 :  야마가타카유키 || 야마가타카유키 -->  일치율:  100 %
 -- 영화이름 :  신배틀로얄:극악소년원2 || 신배틀로얄:극악소년원2 -->  일치율:  100 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
크롤링데이터: 399 신 배틀로얄:극악 소년원2  ->  {'배급사': '', '등급': '[국내] 청소년 관람불가', '장르': '액션', '배우': ''}
--------------------------------------------------
전처리데이터: 400 킬러본색: 블랙페이스  ->  {'배급사': '스크린조이', '등급': '청소년관람불가', '장르': '액션', '배우': '야마구치 요시유키'}
--------------------------------------------------
전처리데이터: 401 닌자걸의 정사  ->  {'배급사': '(유)조이앤컨텐츠그룹', '등급': '청소년관람불가', '장르': '멜로/로맨스', '배우': '사야마 아이'}
--------------------------------------------------
전처리데이터: 402 호스트걸의 비밀섹스  ->  {'배급사': '(유)조이앤컨텐츠그룹', '등급': '청소년관람불가', '장르': '멜로/로맨스', '배우': '나루세 코코미'}
--------------------------------------------------
 -

In [47]:
df1 =pd.read_excel("Distributor20210804175633.xlsx")
df2 =pd.read_excel("Distributor20210804183643.xlsx")
df3 =pd.read_excel("Distributor20210804183700.xlsx")
df4 =pd.read_excel("Distributor20210804185723.xlsx")
df5 =pd.read_excel("Distributor20210804190418.xlsx")

df=pd.concat([df1,df2,df3,df4,df5])

In [51]:
df.to_csv("data5000_10000.csv",encoding = 'utf-8-sig')

In [103]:
count = 0 
for i in range(len(df_sum)):
    if type(df_sum.iloc[i]["배우"]) == float:
    
    

False